# Importing Libraries


In [1]:
from torchvision import models
from torchvision import transforms, datasets, models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torch.cuda.amp import GradScaler, autocast
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from PIL import Image

# Setting up GPU

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')
if torch.cuda.device_count() > 1:
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    model = torch.nn.DataParallel(model)
    

Using device: cuda:0


# Setting up model

In [7]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  
])

In [15]:
dataset_path = 'D:\\Datasets\\CropDiseaseClassificationOriginal\\split_crop_diseasev2\\train' 
testing_path = 'D:\\Datasets\\CropDiseaseClassificationOriginal\\split_crop_diseasev2\\test' 
dataset = ImageFolder(root=dataset_path, transform=transform)
test_dataset = ImageFolder(root=testing_path, transform=transform)
batch_size = 32
num_workers = 4
pin_memory = True

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)

model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)

num_epochs=20

for epoch in range(num_epochs): 
    model.train()  
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  
        optimizer.zero_grad()  
        outputs = model(images)  
        loss = criterion(outputs, labels) 
        loss.backward()  
        optimizer.step()  
        total_loss += loss.item() * images.size(0)
    print(f'Epoch {epoch+1}: Train Loss: {total_loss / len(train_loader.dataset):.4f}')



C:\Users\abdul\miniconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\abdul\miniconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1: Train Loss: 1.2249
Epoch 2: Train Loss: 0.8215
Epoch 3: Train Loss: 0.6062
Epoch 4: Train Loss: 0.3984
Epoch 5: Train Loss: 0.2389
Epoch 6: Train Loss: 0.1266
Epoch 7: Train Loss: 0.0642
Epoch 8: Train Loss: 0.0409
Epoch 9: Train Loss: 0.0266
Epoch 10: Train Loss: 0.0197
Epoch 11: Train Loss: 0.0149
Epoch 12: Train Loss: 0.0146
Epoch 13: Train Loss: 0.0095
Epoch 14: Train Loss: 0.0090
Epoch 15: Train Loss: 0.0103
Epoch 16: Train Loss: 0.0085
Epoch 17: Train Loss: 0.0068
Epoch 18: Train Loss: 0.0073
Epoch 19: Train Loss: 0.0050
Epoch 20: Train Loss: 0.0052


In [20]:
from sklearn.metrics import classification_report

def get_predictions(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return all_labels, all_preds

labels, preds = get_predictions(model, test_loader)
print(classification_report(labels, preds, target_names=['Cassava Bacterial Blight (CBB)', 'Cassava Brown Streak Disease (CBSD)','Cassava Green Mottle (CGM)','Cassava Mosaic Disease (CMD)','Healthy']))

                                     precision    recall  f1-score   support

     Cassava Bacterial Blight (CBB)       0.59      0.56      0.57       221
Cassava Brown Streak Disease (CBSD)       0.73      0.71      0.72       431
         Cassava Green Mottle (CGM)       0.76      0.68      0.72       493
       Cassava Mosaic Disease (CMD)       0.80      0.86      0.83       528
                            Healthy       0.61      0.65      0.63       466

                           accuracy                           0.71      2139
                          macro avg       0.70      0.69      0.69      2139
                       weighted avg       0.71      0.71      0.71      2139



In [22]:
#BEST ONE YET
model = models.resnext50_32x4d(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
num_epochs = 25
for epoch in range(num_epochs): 
    model.train()  
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  
        optimizer.zero_grad()  
        outputs = model(images)  
        loss = criterion(outputs, labels) 
        loss.backward()  
        optimizer.step()  
        total_loss += loss.item() * images.size(0)  
    print(f'Epoch {epoch+1}: Train Loss: {total_loss / len(train_loader.dataset):.4f}')



C:\Users\abdul\miniconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\abdul\miniconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1: Train Loss: 0.8608
Epoch 2: Train Loss: 0.3981
Epoch 3: Train Loss: 0.1768
Epoch 4: Train Loss: 0.1087
Epoch 5: Train Loss: 0.0970
Epoch 6: Train Loss: 0.0861
Epoch 7: Train Loss: 0.0747
Epoch 8: Train Loss: 0.0903
Epoch 9: Train Loss: 0.0704
Epoch 10: Train Loss: 0.0774
Epoch 11: Train Loss: 0.0846
Epoch 12: Train Loss: 0.0557
Epoch 13: Train Loss: 0.0306
Epoch 14: Train Loss: 0.0696
Epoch 15: Train Loss: 0.0714
Epoch 16: Train Loss: 0.0702
Epoch 17: Train Loss: 0.0238
Epoch 18: Train Loss: 0.0491
Epoch 19: Train Loss: 0.0600
Epoch 20: Train Loss: 0.0245
Epoch 21: Train Loss: 0.0254
Epoch 22: Train Loss: 0.0648
Epoch 23: Train Loss: 0.0220
Epoch 24: Train Loss: 0.0140
Epoch 25: Train Loss: 0.0243


In [23]:
labels, preds = get_predictions(model, test_loader)
print(classification_report(labels, preds, target_names=['Cassava Bacterial Blight (CBB)', 'Cassava Brown Streak Disease (CBSD)','Cassava Green Mottle (CGM)','Cassava Mosaic Disease (CMD)','Healthy']))

                                     precision    recall  f1-score   support

     Cassava Bacterial Blight (CBB)       0.56      0.62      0.59       221
Cassava Brown Streak Disease (CBSD)       0.77      0.70      0.73       431
         Cassava Green Mottle (CGM)       0.75      0.80      0.77       493
       Cassava Mosaic Disease (CMD)       0.80      0.86      0.83       528
                            Healthy       0.68      0.60      0.64       466

                           accuracy                           0.73      2139
                          macro avg       0.71      0.72      0.71      2139
                       weighted avg       0.73      0.73      0.73      2139



In [25]:
model = models.resnet152(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
num_epochs = 25
for epoch in range(num_epochs): 
    model.train()  
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  
        optimizer.zero_grad()  
        outputs = model(images)  
        loss = criterion(outputs, labels) 
        loss.backward()  
        optimizer.step()  
        total_loss += loss.item() * images.size(0)  
    print(f'Epoch {epoch+1}: Train Loss: {total_loss / len(train_loader.dataset):.4f}')

Epoch 1: Train Loss: 0.9118
Epoch 2: Train Loss: 0.5199
Epoch 3: Train Loss: 0.2722
Epoch 4: Train Loss: 0.1586


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x0000023D4C47D990>
Traceback (most recent call last):
  File "C:\Users\abdul\miniconda3\envs\py310\lib\site-packages\torch\utils\data\dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "C:\Users\abdul\miniconda3\envs\py310\lib\site-packages\torch\utils\data\dataloader.py", line 1437, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


Epoch 5: Train Loss: 0.1521
Epoch 6: Train Loss: 0.0811
Epoch 7: Train Loss: 0.0703
Epoch 8: Train Loss: 0.1054
Epoch 9: Train Loss: 0.0711
Epoch 10: Train Loss: 0.0781
Epoch 11: Train Loss: 0.0630
Epoch 12: Train Loss: 0.1006
Epoch 13: Train Loss: 0.0479
Epoch 14: Train Loss: 0.0680
Epoch 15: Train Loss: 0.0361
Epoch 16: Train Loss: 0.0539
Epoch 17: Train Loss: 0.1290
Epoch 18: Train Loss: 0.0374
Epoch 19: Train Loss: 0.0216
Epoch 20: Train Loss: 0.0329
Epoch 21: Train Loss: 0.0154
Epoch 22: Train Loss: 0.0181
Epoch 23: Train Loss: 0.0972
Epoch 24: Train Loss: 0.0849
Epoch 25: Train Loss: 0.0619


In [26]:
labels, preds = get_predictions(model, test_loader)
print(classification_report(labels, preds, target_names=['Cassava Bacterial Blight (CBB)', 'Cassava Brown Streak Disease (CBSD)','Cassava Green Mottle (CGM)','Cassava Mosaic Disease (CMD)','Healthy']))

                                     precision    recall  f1-score   support

     Cassava Bacterial Blight (CBB)       0.52      0.56      0.54       221
Cassava Brown Streak Disease (CBSD)       0.83      0.66      0.73       431
         Cassava Green Mottle (CGM)       0.78      0.68      0.73       493
       Cassava Mosaic Disease (CMD)       0.81      0.84      0.82       528
                            Healthy       0.59      0.75      0.66       466

                           accuracy                           0.72      2139
                          macro avg       0.71      0.70      0.70      2139
                       weighted avg       0.73      0.72      0.72      2139

